<a href="https://colab.research.google.com/github/PradHolla/TF-Recommenders/blob/main/Notebook/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 4.0 MB 5.5 MB/s 


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import numpy as np
from typing import Dict, Text
import pprint

In [3]:
# ratings data
rating = tfds.load('movielens/100k-ratings', split='train')
# features of all the movies
movies = tfds.load('movielens/100k-movies', split='train')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling movielens-train.tfrecord...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.0. Subsequent calls will reuse this data.


In [4]:
# limiting the features
rating = rating.map(lambda x:{'movie_title':x['movie_title'],'user_id':x['user_id']})
movies = movies.map(lambda x: x['movie_title'])

In [5]:
user_id_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_id_vocabulary.adapt(rating.map(lambda x: x['user_id']))

In [6]:
movies_title_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movies_title_vocabulary.adapt(movies)

In [7]:
class MovieLensModel(tfrs.Model):
 
  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()
 
    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model
 
    # Set up a retrieval task.
    self.task = task
 
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.
 
    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])
 
    return self.task(user_embeddings, movie_embeddings)

In [8]:
users_model = tf.keras.Sequential([user_id_vocabulary,
                                   tf.keras.layers.Embedding(user_id_vocabulary.vocabulary_size(),64)])
movie_model = tf.keras.Sequential([movies_title_vocabulary,                                   tf.keras.layers.Embedding(movies_title_vocabulary.vocabulary_size(),64)])
 
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)))

In [9]:
model = MovieLensModel(users_model,movie_model,task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
model.fit(rating.batch(4096), epochs=3)

Epoch 1/3
25/25 [==============================] - 18s 590ms/step - factorized_top_k/top_1_categorical_accuracy: 1.2000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0048 - factorized_top_k/top_50_categorical_accuracy: 0.0441 - factorized_top_k/top_100_categorical_accuracy: 0.0986 - loss: 33071.2919 - regularization_loss: 0.0000e+00 - total_loss: 33071.2919
Epoch 2/3
25/25 [==============================] - 15s 587ms/step - factorized_top_k/top_1_categorical_accuracy: 1.4000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0049 - factorized_top_k/top_10_categorical_accuracy: 0.0141 - factorized_top_k/top_50_categorical_accuracy: 0.1048 - factorized_top_k/top_100_categorical_accuracy: 0.2110 - loss: 31003.5520 - regularization_loss: 0.0000e+00 - total_loss: 31003.5520
Epoch 3/3
25/25 [==============================] - 15s 600ms/step - factorized_top_k/top_1_categorical_accuracy: 3.2000e-04 - factorized_top_k/top_5_categor

In [10]:
recommends = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
recommends.index_from_dataset(movies.batch(100).map(lambda title: (title, model.movie_model(title))))

In [12]:
id_ = input('Enter the user_id: ')
_, titles = recommends(np.array([str(id_)]))
print('Top recommendation for user',id_,titles[0, :3])

Enter the user_id: 11
Top recommendation for user 11 tf.Tensor(
[b'Mary Reilly (1996)'
 b'Englishman Who Went Up a Hill, But Came Down a Mountain, The (1995)'
 b'Go Fish (1994)'], shape=(3,), dtype=string)


In [22]:
path = '/content/tf_rec_model'
tf.saved_model.save(recommends, path)

INFO:tensorflow:Assets written to: /content/tf_rec_model/assets


INFO:tensorflow:Assets written to: /content/tf_rec_model/assets


In [23]:
loaded = tf.saved_model.load(path)

# Pass a user id in, get top predicted movie titles back.
scores, titles = loaded(["111"])

print(f"Recommendations: {titles[0][:3]}")

Recommendations: [b'Kolya (1996)' b'Full Monty, The (1997)' b'Sweet Hereafter, The (1997)']


In [26]:
!tar -czvf "/content/tf_rec_model.tar.gz" "/content/tf_rec_model"

tar: Removing leading `/' from member names
/content/tf_rec_model/
/content/tf_rec_model/assets/
/content/tf_rec_model/saved_model.pb
/content/tf_rec_model/variables/
/content/tf_rec_model/variables/variables.data-00000-of-00001
/content/tf_rec_model/variables/variables.index
